## Setup

In [ ]:
import splusdata
import pandas as pd
from astropy.io import fits
from dl import queryClient as qc
from config_loader import load_config

In [3]:
template_field_names = ["STRIPE82-0007",
                        "STRIPE82-0008",
                        "STRIPE82-0009",
                        "STRIPE82-0010",
                        "STRIPE82-0011",
                        "STRIPE82-0012",
                        "STRIPE82-0013",
                        "STRIPE82-0014",
                        "STRIPE82-0015",
                        "STRIPE82-0016",
                        "STRIPE82-0017",
                        "STRIPE82-0018",
                        "STRIPE82-0019",
                        "STRIPE82-0020"]

BANDS = ["g",
         "J0515",
         "r",
         "i",
         "J0861",
         "z",
         "J0660", 
         "u",
         "J0378",
         "J0395", 
         "J0410", 
         "J0430"]

In [5]:
paths = load_config("config/paths.json")
auths = load_config(paths["config_path"] + "/" + "auth.json")

In [ ]:
login = auths["login"]
pwd = auths["password"]

conn = splusdata.Core(login, pwd)

## Get tables

In [4]:
fields_coords = pd.read_csv(paths['tables_path'] + "/" + "iDR4_obs_seeing.csv")
fields_coords.Field = fields_coords.Field.replace({'_': '-', ' ': ''}, regex=True)

In [ ]:
query_fields = ""
for field in template_field_names:
    query_fields += field + "|"
fields_coords = fields_coords[fields_coords.Field.str.contains(query_fields[:-1])]
fields_coords = fields_coords.set_index('Field')

In [ ]:
for field in template_field_names:
    field_name = field
    ra, dec = fields_coords.loc[field_name].RA, fields_coords.loc[field_name].DEC
    offset = 0.7
    ra0 = ra - offset
    ra1 = ra + offset
    dec0 = dec - offset
    dec1 = dec + offset

    dual = qc.query(sql=f"SELECT id, fwhm_n, r_auto, r_petro, r_iso FROM splus_dr4.dual WHERE q3c_poly_query(ra, dec, ARRAY[{ra0}, {dec0}, {ra0}, {dec1}, {ra1}, {dec1}, {ra1}, {dec0}])", fmt="pandas")
    objs = qc.query(sql=f"SELECT id, ra, dec, class, prob_qso, prob_star, prob_gal FROM splus_dr4.sqg WHERE q3c_poly_query(ra, dec, ARRAY[{ra0}, {dec0}, {ra0}, {dec1}, {ra1}, {dec1}, {ra1}, {dec0}])", fmt="pandas")
    objs = pd.merge(objs, dual, on="id")
    objs.to_csv(paths['tables_paths'] + "/" + f"DR4-{field_name}-objs.csv", index=False)

## Get fields

In [ ]:
for field_name in template_field_names:
    for band in BANDS:
        hdus = conn.field_frame(field_name, band)
        filename = field_name + "-" + band + ".fits"
        hdus.writeto(paths['data_path'] + "/" + filename, overwrite=False)